In [ ]:
import sys
import json
import copy
with open('./paths', 'r') as f:
    globals().update(json.loads(f.read()))
sys.path.append(path_Modules_Product_)
sys.path.append(path_Modules_Toolbox_)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import data_processing as myprep
import handles as myhandles
import plots as myplots

# Data Dist.

## dataset_main

In [ ]:
dataset_ = pd.read_csv('../../Data/processed/dataset/ds_main.csv', index_col=0)
cancer_sets = ['Colorectum', 'Lung', 'Breast', 'Ovary', 'Stomach', 'Esophagus']
normal_set = 'Normal'
all_sets = copy.deepcopy(cancer_sets)
all_sets.append(normal_set)
dataset = dataset_.dropna()
dataset = dataset.loc[np.logical_and(dataset.s_Age >= 40,dataset.s_Age <= 70)]
dataset = dataset.loc[np.isin(dataset.s_Tumor, all_sets)].copy()
dataset['Target'] = 1
dataset.loc[dataset.s_Tumor == 'Normal', 'Target'] = 0

In [ ]:
n_fig_rows = np.ceil(len(p_markers))
plt.figure(figsize=(15, 5 * n_fig_rows))
for c, p in enumerate(p_markers):
    ax = plt.subplot(n_fig_rows, 2, 2 * c + 1)
    for t, dt in dataset.groupby('s_Tumor'):
        pdata = np.log10(dt[p])
        i = np.linspace(pdata.min(), pdata.max(), 100)
        pdata.plot.kde(ax=ax, label=t, ind=i, grid=True)
    plt.title(p)
    plt.legend()
    ax = plt.subplot(n_fig_rows, 2, 2 * c + 2)
    for t, dt in dataset.groupby('Target'):
        pdata = np.log10(dt[p])
        i = np.linspace(pdata.min(), pdata.max(), 100)
        pdata.plot.kde(ax=ax, label=t, ind=i, grid=True)
    plt.title(p)
    plt.legend()

## OOB B

In [ ]:
ds = pd.read_csv('../Data/processed/dataset/ds_OOB_B.csv',index_col=0)
ds

In [ ]:
np.sort([x for x in ds.columns if x.startswith('p_')])

In [ ]:
ax=np.log10(ds['p_CA15-3_1']).plot.kde()
np.log10(ds['p_CA15-3_2']).plot.kde(ax=ax)

## OOB C

In [ ]:
ds = pd.read_csv('../Data/processed/dataset/ds_OOB_C.csv',index_col=0)
ds

### Proteins

In [ ]:
np.sort([x for x in ds.columns if x.startswith('p_')])

In [ ]:
m = [ 'p_PROLACTIN_L',
       'p_PROLACTIN_M','p_PROLACTIN']
fig = plt.figure(figsize=(15, 5))
ax = plt.subplot(1, 2 ,1)
np.log10(ds[m].dropna()).plot.kde(ax=ax)
ax = plt.subplot(1, 2 ,2)
plt.axis('off')
plt.text(0,0,ds[m].describe().to_string());

### Phenotypes

In [ ]:
i = 's_BMI'
d_ = ds[i]
d_.plot.kde(ind = np.linspace(d_.min(),d_.max(),100))

### Other

In [ ]:
ds.s_Smoker.value_counts()

# Normalization

## OOB C

In [ ]:
ds = pd.read_csv('../Data/processed/dataset/ds_OOB_C.csv',index_col=0)
ds

### log -> standard

# Phenotype Corr.

## dataset_main

In [ ]:
dataset = pd.read_csv('../Data/processed/dataset/ds_main.csv',index_col=0)
p_markers = [x for x in dataset.columns if x.startswith('p_')]

In [ ]:
ds = dataset.loc[dataset.s_Tumor == 'Normal'].copy()
i_male = ds.loc[ds.s_Sex == 'M'].index
i_female = ds.loc[ds.s_Sex == 'F'].index

In [ ]:
fig_cols = int(np.ceil(len(p_markers) / 2))
fig = myhandles.default_fig(figsize=(15, 5 * fig_cols))
age_bins = np.arange(20, 81, 10)
age_sets = age_bins + 5
for c1, p in enumerate(p_markers):
    ax = plt.subplot(fig_cols, 2, c1 + 1)
    for i, g, c in zip([i_male, i_female], ['male', 'female'], ['b', 'r']):
        d = ds.loc[i]
#         d = d.drop(d.loc[d[p + '-lim'] != 0].index)
        d.plot.scatter(x='s_Age', y=p, label=g, ax=ax, color=c, alpha=.6)
        dsc = np.empty((len(age_bins), 2))*np.nan
        digz = np.digitize(d.s_Age, age_bins)
        for c2, d2 in enumerate(d.groupby(digz)):
            c3 = d2[0]
            if c3 == len(age_bins):
                break
            d2 = d2[1]
            dsc[c2, 0] = age_sets[c3]
            dsc[c2, 1] = d2[p].median()
        plt.plot(dsc[:, 0], dsc[:, 1], c)
    plt.ylabel(c1)
    plt.xlabel('')
    plt.grid()
plt.tight_layout()